In [1]:
import pandas as pd
import numpy as np

## Import all 3 sources of data

In [254]:
vhdb = pd.read_csv("Virus_Host_DB.txt", delimiter="\t", header=None, names=['phage', 'host'])
vhdb.head(1)

,phage,host
0,NC_001447,NC_010163.1


In [267]:
phnew = pd.read_csv("phages_to_host.csv", header=0)
phnew = phnew[['phage_genbank_id','phage_refseq_id', 'host_id']]
phnew = phnew.rename(columns={"phage_genbank_id": "phage", 
                              "host_id": "host", 
                              "phage_refseq_id":"refseq"})
phnew.drop(phnew[phnew['host'] == 'Not Found'].index, inplace = True) 
phnew.head(1)

,phage,refseq,host
0,KJ410132,NC_023597,MT818425.1


In [256]:
CNNtr = pd.read_csv("training_set.csv", delimiter="\t")
CNNte = pd.read_csv("test_set.csv", delimiter="\t")
t1    = CNNtr[CNNtr['class'] == 1]
t2    = CNNte[CNNte['class'] == 1]
CNN   = pd.concat([t1, t2])
CNN.head(1)

,phage,host,class
20,NC_026583,CP027541,1


## Basic Statistics

In [268]:
vhdbhosts  = set(vhdb['host'].to_numpy())
CNNhosts   = set(CNN['host'].to_numpy())
phnewhosts = set(phnew['host'].to_numpy())     # INCORRECT

In [269]:
vhdbphage  = set(vhdb['phage'].to_numpy())
CNNphage   = set(CNN['phage'].to_numpy())
phnewphage = set(phnew['phage'].to_numpy())

In [270]:
print("{} Pairs, {} unique phage, {} unique host in Virus-Host DB".format(vhdb.shape, len(vhdbphage), len(vhdbhosts)))
print("{} Pairs, {} unique phage, {} unique host in PhageDB".format(phnew.shape, len(phnewphage), len(phnewhosts)))
print("{} Pairs, {} unique phage, {} unique host in CNN paper data".format(CNN.shape, len(CNNphage), len(CNNhosts)))

(1707, 2) Pairs, 1679 unique phage, 261 unique host in Virus-Host DB
(3469, 3) Pairs, 3179 unique phage, 79 unique host in PhageDB
(3469, 3) Pairs, 3449 unique phage, 301 unique host in CNN paper data


<p style="color:blue"> Because we see that most phages are unique, we can use it as foreign key to deduplicate phage-host pair. Also, PhageDB has inaccurate host id; CNN paper compiled PhageDB data back in March 2019 (so there should be a sufficient amount of overlap); Virus-Host DB data are all RefSeq (which is better than Genebank id since RefSeq is maintained by NCBI and all the RefSeq id for the bacteria are supposedly "representative") <p/>

## Merge

In [271]:
def merge_into(dbA, dbB):
    """ We will merge database B into 
        database A by leveraging the 
        fact that most phages occurred 
        only once (and with a unique ID)
        So we essentially are unifying 
        the bacteria (host) accession number.
    """
    
    phage_map, host_map, suspicious = {}, {}, {}
    
    for phage in dbA['phage']:
        a = dbA[dbA['phage'] == phage]
        b = dbB[dbB['phage'] == phage]
        if not len(b) and dbB.shape[1] == 3:
            b = dbB[dbB['refseq'] == phage]

        # limit us to only one-to-one mapping
        if len(a) == len(b) and len(b) == 1:
            anumber = a['host'].to_numpy()[0]
            bnumber = b['host'].to_numpy()[0]
            
            if bnumber in suspicious:
                suspicious[bnumber].add(anumber)
                continue
            elif bnumber not in host_map:
                host_map[bnumber] = anumber
            elif (bnumber in host_map and 
                  host_map[bnumber] != anumber):
                r = host_map.pop(bnumber)
                suspicious[bnumber] = set({r, anumber})
            
            phage_map[b['phage'].to_numpy()[0]] = phage
            
        # If duplicates exist in dbB
        if len(a) > 1 and len(b) > 1:
            print(a.to_numpy())
            print(b.to_numpy())
           
    return phage_map, host_map, suspicious 

#### Merge PhageDB into CNN First

In [272]:
phage_map, host_map, suspicious = merge_into(CNN, phnew)

In [273]:
suspicious

{'MN294681.1': {'CP019892', 'NZ_QTTZ01000001'},
 'LT708304.1': {'CP027541', 'NZ_CM001149'}}

In [275]:
for k,v in host_map.items():
    phnew = phnew.replace(k, v)
for k,v in phage_map.items():
    phnew = phnew.replace(k, v)
cp_ = pd.concat((CNN[['phage', 'host']], 
                 phnew[['phage', 'host']]), axis=0, ignore_index=True)
cp_combined = cp_.drop_duplicates()
cp_combined.shape

(4731, 2)

#### Merge CP combined into VirusHost DB

In [276]:
phage_map, host_map, suspicious = merge_into(vhdb, cp_combined)

[['NC_004687' 'NZ_LN831039.1']
 ['NC_004687' 'NZ_CP011491.1']]
[['NC_004687' 'CP027541']
 ['NC_004687' 'CP020809']]
[['NC_004687' 'NZ_LN831039.1']
 ['NC_004687' 'NZ_CP011491.1']]
[['NC_004687' 'CP027541']
 ['NC_004687' 'CP020809']]
[['NC_003387' 'NZ_AP012555.1']
 ['NC_003387' 'NZ_LN831039.1']]
[['NC_003387' 'CP027541']
 ['NC_003387' 'CP018363']]
[['NC_003387' 'NZ_AP012555.1']
 ['NC_003387' 'NZ_LN831039.1']]
[['NC_003387' 'CP027541']
 ['NC_003387' 'CP018363']]


In [280]:
for k,v in host_map.items():
    cp_combined = cp_combined.replace(k, v)
for k,v in phage_map.items():
    cp_combined = cp_combined.replace(k, v)
combined = pd.concat((cp_combined[['phage', 'host']], 
                 vhdb[['phage', 'host']]), axis=0, ignore_index=True)
combined = combined.drop_duplicates()
combined.shape

(5688, 2)

### Verify Merging

In [284]:
from Bio import SeqIO, Entrez
import pandas as pd
import time

Entrez.email = "garycloudyang@gmail.com"

#### Extract host taxonomy information

In [334]:
hs = set(combined['host'].to_list())

In [287]:
df = pd.DataFrame(columns=['accn', 'Name', 'Taxonomy'])
row = 0

for h in hs:
    handle = Entrez.efetch(db="nuccore", id=h, rettype="gb", retmode="xml")
    res = Entrez.read(handle)
    handle.close()
    df.loc[row] =  [h] + [res[0]['GBSeq_organism']] + [res[0]['GBSeq_taxonomy']]
    print(res[0]['GBSeq_taxonomy'])
    print(res[0]['GBSeq_organism'])
    row += 1
    time.sleep(0.1)

Bacteria; Chlamydiae; Chlamydiales; Chlamydiaceae; Chlamydia/Chlamydophila group; Chlamydia
Chlamydia pneumoniae LPCoLN
Archaea; Euryarchaeota; Stenosarchaea group; Halobacteria; Halobacteriales; Haloarculaceae; Haloarcula
Haloarcula hispanica N601
Bacteria; Proteobacteria; Gammaproteobacteria; Vibrionales; Vibrionaceae; Vibrio
Vibrio alginolyticus
Bacteria; Proteobacteria; Gammaproteobacteria; Enterobacterales; Enterobacteriaceae; Leclercia
Leclercia adecarboxylata
Bacteria; Proteobacteria; Alphaproteobacteria; Rhodobacterales; Rhodobacteraceae; Ruegeria
Ruegeria pomeroyi DSS-3
Bacteria; Proteobacteria; Gammaproteobacteria; Pseudomonadales; Pseudomonadaceae; Pseudomonas
Pseudomonas sp.
Bacteria; Proteobacteria; Gammaproteobacteria; Vibrionales; Vibrionaceae; Salinivibrio
Salinivibrio costicola
Bacteria; Proteobacteria; Gammaproteobacteria; Xanthomonadales; Xanthomonadaceae; Xanthomonas
Xanthomonas oryzae pv. oryzicola
Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcaceae; Staph

In [335]:
# If wish to save to csv
# df.to_csv("Host_taxonomy.csv")

In [305]:
# Number of Hosts that are Eukaryota
df[df['Taxonomy'].str.startswith('E')].shape

,accn,Name,Taxonomy
24,KY464958,Apis mellifera lamarckii,Eukaryota; Metazoa; Ecdysozoa; Arthropoda; Hex...
39,KC900889,Phaeocystis globosa,Eukaryota; Haptista; Haptophyta; Prymnesiophyc...
44,KT970065,Tipula cockerelliana,Eukaryota; Metazoa; Ecdysozoa; Arthropoda; Hex...
45,NC_014574,Culex quinquefasciatus,Eukaryota; Metazoa; Ecdysozoa; Arthropoda; Hex...
47,KP691602,Dunaliella viridis,Eukaryota; Viridiplantae; Chlorophyta; core ch...
56,GG663735,Micromonas pusilla CCMP1545,Eukaryota; Viridiplantae; Chlorophyta; Mamiell...
96,NC_001700,Felis catus,Eukaryota; Metazoa; Chordata; Craniata; Verteb...
99,HQ914635,Chlorella variabilis,Eukaryota; Viridiplantae; Chlorophyta; core ch...
101,EU557093,Tursiops truncatus,Eukaryota; Metazoa; Chordata; Craniata; Verteb...
126,MF925712,Equus caballus,Eukaryota; Metazoa; Chordata; Craniata; Verteb...


In [295]:
# Number of Hosts that are Archea
df[df['Taxonomy'].str.startswith('A')].shape

(13, 3)

In [296]:
# Number of Hosts that are bacteria
df[df['Taxonomy'].str.startswith('B')].shape

(369, 3)

In [307]:
# Hosts that are not bacteria
to_exclude = df[~df['Taxonomy'].str.startswith('B')].accn.to_list()

In [337]:
# Remaining hosts
filtered = combined[~combined['host'].isin(to_exclude)]
df[df['Taxonomy'].str.startswith('B')]

,accn,Name,Taxonomy
0,CP001713,Chlamydia pneumoniae LPCoLN,Bacteria; Chlamydiae; Chlamydiales; Chlamydiac...
2,NZ_CP042449.1,Vibrio alginolyticus,Bacteria; Proteobacteria; Gammaproteobacteria;...
3,NZ_CP013990.1,Leclercia adecarboxylata,Bacteria; Proteobacteria; Gammaproteobacteria;...
4,CP000031,Ruegeria pomeroyi DSS-3,Bacteria; Proteobacteria; Alphaproteobacteria;...
5,PEKX01000007,Pseudomonas sp.,Bacteria; Proteobacteria; Gammaproteobacteria;...
...,...,...,...
404,NZ_SJSX01000001.1,Aliivibrio fischeri,Bacteria; Proteobacteria; Gammaproteobacteria;...
405,NZ_LN831039.1,Mycolicibacterium smegmatis,Bacteria; Actinobacteria; Corynebacteriales; M...
406,NZ_CP029614.1,Lactobacillus johnsonii,Bacteria; Firmicutes; Bacilli; Lactobacillales...
407,NZ_CP009610.1,Haemophilus influenzae,Bacteria; Proteobacteria; Gammaproteobacteria;...


In [341]:
len([i.split(";")[-1] for i in df[df['Taxonomy'].str.startswith('B')].Taxonomy.to_list() ])

369

In [344]:
species = [i.split(";")[-1] for i in df[df['Taxonomy'].str.startswith('B')].Taxonomy.to_list()]
keys = set(species)
libs = {}

for s in species:
    if s in libs:
        libs[s] += 1
    else:
        libs[s] = 1

print("{} unique species".format(len(libs)))
{k: v for k, v in sorted(libs.items(), key=lambda item: item[1], reverse=True)}

162 unique species


{' Streptomyces': 28,
 ' Streptococcus': 13,
 ' Vibrio': 11,
 ' Pseudomonas': 11,
 ' Gordonia': 10,
 ' Staphylococcus': 9,
 ' Microbacterium': 9,
 ' Corynebacterium': 7,
 ' Salmonella': 7,
 ' Lactobacillus': 6,
 ' Pseudoalteromonas': 6,
 ' Brucella': 6,
 ' Xanthomonas': 5,
 ' Bacillus': 5,
 ' Erwinia': 5,
 ' Bacillus cereus group': 5,
 ' Mycolicibacterium': 5,
 ' Burkholderia cepacia complex': 5,
 ' Clostridium': 4,
 ' Nocardia': 4,
 ' Spiroplasma': 4,
 ' Rhodococcus': 4,
 ' Chlamydia': 3,
 ' Pectobacterium': 3,
 ' Acinetobacter': 3,
 ' Yersinia': 3,
 ' Rhizobium': 3,
 ' Mycobacterium avium complex (MAC)': 3,
 ' Brevibacterium': 3,
 ' Arthrobacter': 3,
 ' Lactiplantibacillus': 3,
 ' Shigella': 3,
 ' Enterobacter cloacae complex': 3,
 ' Aeromonas': 3,
 ' Serratia': 3,
 ' Klebsiella': 3,
 ' Lacticaseibacillus': 3,
 ' Tsukamurella': 3,
 ' Enterococcus': 3,
 ' Flavobacterium': 3,
 ' Edwardsiella': 2,
 ' Cronobacter': 2,
 ' Rhodobacter': 2,
 ' Sinorhizobium': 2,
 ' Ralstonia': 2,
 ' Leucono

In [345]:
# Unique phage accession
len(set(filtered['phage']))

5446

In [346]:
# Total phage accession
len(filtered['phage'])

5622

<p style="color:blue"> Given there are close to 400 hosts, yet only about 160 unique species, we might wonder if we should aggregate all these species together to produce a more uniform dataset. I decided not to, since there are 5446 out of 5622 unique phage accession numbers, which means that viruses are infecting different strands of the bacteria in the same species.<p/>

In [383]:
filtered.to_csv("Combined2.csv")

#### Not Used

In [381]:
multiple_host = set()

tmp = filtered.groupby('phage')['host'].apply(list).reset_index(name='new')
for i in range(tmp['new'].to_numpy().shape[0]):
    row = tmp['new'].to_numpy()[i]
    if len(row) > 1:
        multiple_host.add(tuple(row))
        
multiple_host

{('AL645882', 'NZ_BBQG01000095.1'),
 ('AL645882', 'NZ_JAANOB010000217.1'),
 ('AL645882', 'NZ_MDFG01000001.1'),
 ('CP000388', 'NZ_CP011028.1'),
 ('CP000388', 'NZ_CP023464.1'),
 ('CP019892', 'MN294681.1'),
 ('CP024902', 'NZ_CP008781.1', 'NZ_CP024902.1'),
 ('JTHH01000008', 'NZ_CM000747.1'),
 ('JTHH01000008', 'NZ_CP011007.1'),
 ('JTHH01000008', 'NZ_CP053931.1', 'NZ_CM000747.1'),
 ('NC_003155.5', 'NC_016111.1'),
 ('NC_007530.2', 'NZ_CP053931.1'),
 ('NC_013961.1', 'NZ_CP023567.1'),
 ('NC_021505.1', 'NZ_CP048408.1'),
 ('NZ_CABEIC010000001.1',
  'NZ_CP041695.1',
  'NC_012490.1',
  'NZ_QEOC01000001.1',
  'NC_018681.1'),
 ('NZ_CABEIC010000001.1',
  'NZ_LR134352.1',
  'NZ_CP006850.1',
  'NZ_AWTB01000001.1',
  'NZ_BAOP01000115.1'),
 ('NZ_CP013197.1', 'NZ_AMGI01000001.1'),
 ('NZ_CP027793.1', 'LT708304.1'),
 ('NZ_CP032757.1', 'NZ_CP030105.1'),
 ('NZ_CP045235.1', 'NZ_MUTR01000001.1'),
 ('NZ_CP048408.1',
  'NC_021505.1',
  'NZ_LS999521.1',
  'NC_010554.1',
  'NZ_AP014524.1'),
 ('NZ_LN831039.1', 'CP018